In [9]:
from numpy.random import seed
import pandas as pd
import numpy as np
from sklearn.preprocessing import minmax_scale
# from keras_diagram import ascii
from keras.layers.convolutional import Conv2D, MaxPooling2D, Conv1D, MaxPooling1D
from keras.optimizers import SGD
from keras.models import Sequential
from keras.layers import Dense, Flatten
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support
import keras
import math
from sklearn.metrics import roc_curve, auc, roc_auc_score

data_raw = pd.read_csv("creditcard.csv")



In [10]:
# Assign variables x and y corresponding to row data and it's class value
X = data_raw.loc[:, data_raw.columns != 'Class']
y = data_raw.loc[:, data_raw.columns == 'Class']

In [11]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

def generate_train_test_timeseries(data, test_ratio=0.3):
#     cutoff = math.floor(X.shape[0] *cuttoff_percent) 
#     print cutoff
    total_samples = data.shape[0]
    # splitting index
    cutoff = int(total_samples * (1 - test_ratio))

    
    data.sort_values('Time', inplace=True)
    
    X_train = data.loc[0:cutoff, data.columns != 'Class']
    y_train = data.loc[0:cutoff, data.columns == 'Class']
    X_test = data.loc[cutoff:, data.columns != 'Class']
    y_test = data.loc[cutoff:, data.columns == 'Class']
    
    pipeline = Pipeline([
        ('scaling', StandardScaler()),
    ])
    preprocessor = pipeline.fit(X_train)
    X_train_prp = preprocessor.transform(X_train)
    

    X_test_prp = preprocessor.transform(X_test)
    
    return X_train_prp, y_train, X_test_prp, y_test
    
########################################################################

In [12]:
def reshape_to_batches(a, batch_size):
    # pad with zeros if the length is not divisible by the batch_size
    batch_num = np.ceil((float)(a.shape[0]) / batch_size)
    modulo = batch_num * batch_size - a.shape[0]
    if modulo != 0:
        pad = np.zeros((int(modulo), a.shape[1]))
        a = np.vstack((a, pad))
    return np.array(np.split(a, batch_num))

In [13]:
# a function to reshape batches into the original shape
def _3d_to_2d(arr):
    return arr.reshape(arr.shape[0] * arr.shape[1], arr.shape[2])

In [14]:
def create_model(batch_size=100, window_height=5):
    #import all dependencies
    from keras.layers import Input, Dense, Conv1D
    from keras.models import Model

    seed(2018)
    
    inputs = Input(shape=(batch_size, 30)) # This returns a tensor
    
    # a layer instance is callable on a tensor, and returns a tensor
    conv1 = Conv1D(32, (window_height), # 32 filters with a window of width 5
    strides=1, # think autoregression
    padding='causal', # forward in time
    )(inputs) # syntax to chain layers: Layer(...)(PreviousLayer)
    
    fc1 = Dense(64, activation='relu')(conv1)
    predictions = Dense(2, activation='softmax')(fc1)
    
    model = Model(inputs=inputs,
    outputs=predictions)
    model.compile(optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy'])
    
    return model

In [15]:
def time_series_cross_val(batch_size=100, window_height=5):
    
    precision = []
    recall = []
    f1score = []
    aucs = []
    
    cv = 0.3
    for i in range(3):
        print('Cross validating epoch ', i)
        X_train, y_train, X_test, y_test = generate_train_test_timeseries(data_raw, test_ratio = cv)

        y_test = keras.utils.to_categorical(y_test)
        y_train = keras.utils.to_categorical(y_train)

        X_train_batch = reshape_to_batches(X_train, batch_size)
        y_train_batch = reshape_to_batches(y_train, batch_size)
        y_test_batch = reshape_to_batches(y_test, batch_size)
        X_test_batch = reshape_to_batches(X_test, batch_size)

        print X_train_batch.shape
        print X_test_batch.shape
        print y_train_batch.shape
        print y_test_batch.shape

        model = create_model(batch_size, window_height )
        model.fit(X_train_batch, y_train_batch, epochs=30)
        y_pred = model.predict(X_test_batch)

        # store the raw predictions in a copy
        y_hat = np.copy(y_pred)

        # Classification reporting
        from sklearn.metrics import confusion_matrix, classification_report
        cutt_off_tr = 0.5
        y_pred[np.where(y_pred>=cutt_off_tr)] = 1
        y_pred[np.where(y_pred<cutt_off_tr)]  = 0

        y_pred_2d = _3d_to_2d(y_pred)[:,1]
        y_test_2d = _3d_to_2d(y_test_batch)[:,1]

        print(confusion_matrix(
        _3d_to_2d(y_test_batch)[:, 1],
        _3d_to_2d(y_pred)[:, 1]))

        prfs = precision_recall_fscore_support(y_test_2d, y_pred_2d, labels=[1])

        precision.append(prfs[0])
        recall.append(prfs[1])
        f1score.append(prfs[2])
        aucs.append(roc_auc_score(y_test_2d, _3d_to_2d(y_hat)[:, 1]))

        print prfs
        
        print(classification_report(
        y_test_2d,
        y_pred_2d,
        target_names = ["Genuine", "Fraud"],
        digits = 5))
        
        cv = cv - 0.1
        
    
    return [np.mean(f1score),np.mean(precision),np.mean(recall),np.mean(aucs)]

In [16]:
results_1 = time_series_cross_val(100, 5)
results_1

('Cross validating epoch ', 0)
(1994, 100, 30)
(855, 100, 30)
(1994, 100, 2)
(855, 100, 2)
Epoch 1/30
1994/1994 [==============================] - 2s 988us/step - loss: 0.1893 - acc: 0.9461
Epoch 2/30
1994/1994 [==============================] - 1s 443us/step - loss: 0.0169 - acc: 0.9988
Epoch 3/30
1994/1994 [==============================] - 1s 449us/step - loss: 0.0094 - acc: 0.9989
Epoch 4/30
1994/1994 [==============================] - 1s 453us/step - loss: 0.0071 - acc: 0.9990
Epoch 5/30
1994/1994 [==============================] - 1s 473us/step - loss: 0.0059 - acc: 0.9990
Epoch 6/30
1994/1994 [==============================] - 1s 457us/step - loss: 0.0052 - acc: 0.9991
Epoch 7/30
1994/1994 [==============================] - 1s 466us/step - loss: 0.0047 - acc: 0.9992
Epoch 8/30
1994/1994 [==============================] - 1s 483us/step - loss: 0.0043 - acc: 0.9992
Epoch 9/30
1994/1994 [==============================] - 1s 490us/step - loss: 0.0040 - acc: 0.9992
Epoch 10/30
1994/1

2564/2564 [==============================] - 1s 411us/step - loss: 0.0031 - acc: 0.9994
Epoch 14/30
2564/2564 [==============================] - 1s 399us/step - loss: 0.0030 - acc: 0.9994
Epoch 15/30
2564/2564 [==============================] - 1s 388us/step - loss: 0.0031 - acc: 0.9993
Epoch 16/30
2564/2564 [==============================] - 1s 393us/step - loss: 0.0027 - acc: 0.9995
Epoch 17/30
2564/2564 [==============================] - 1s 394us/step - loss: 0.0027 - acc: 0.9994
Epoch 18/30
2564/2564 [==============================] - 1s 398us/step - loss: 0.0025 - acc: 0.9995
Epoch 19/30
2564/2564 [==============================] - 1s 412us/step - loss: 0.0024 - acc: 0.9995
Epoch 20/30
2564/2564 [==============================] - 1s 379us/step - loss: 0.0023 - acc: 0.9995
Epoch 21/30
2564/2564 [==============================] - 1s 396us/step - loss: 0.0022 - acc: 0.9995
Epoch 22/30
2564/2564 [==============================] - 1s 406us/step - loss: 0.0022 - acc: 0.9995
Epoch 23/30


[0.6748871465906392,
 0.9345335515548282,
 0.53024691358024689,
 0.95273114292797467]

In [17]:
results_2 = time_series_cross_val(200, 5)
results_2

('Cross validating epoch ', 0)
(997, 200, 30)
(428, 200, 30)
(997, 200, 2)
(428, 200, 2)
Epoch 1/30
997/997 [==============================] - 2s 2ms/step - loss: 0.3242 - acc: 0.8953
Epoch 2/30
997/997 [==============================] - 1s 900us/step - loss: 0.0470 - acc: 0.9982
Epoch 3/30
997/997 [==============================] - 1s 792us/step - loss: 0.0184 - acc: 0.9988
Epoch 4/30
997/997 [==============================] - 1s 827us/step - loss: 0.0118 - acc: 0.9989
Epoch 5/30
997/997 [==============================] - 1s 837us/step - loss: 0.0091 - acc: 0.9990
Epoch 6/30
997/997 [==============================] - 1s 839us/step - loss: 0.0076 - acc: 0.9990
Epoch 7/30
997/997 [==============================] - 1s 815us/step - loss: 0.0067 - acc: 0.9990
Epoch 8/30
997/997 [==============================] - 1s 784us/step - loss: 0.0060 - acc: 0.9990
Epoch 9/30
997/997 [==============================] - 1s 911us/step - loss: 0.0055 - acc: 0.9991 0s - loss: 0.0056 - acc:
Epoch 10/30
997

1282/1282 [==============================] - 1s 826us/step - loss: 0.0044 - acc: 0.9992
Epoch 13/30
1282/1282 [==============================] - 1s 819us/step - loss: 0.0041 - acc: 0.9992
Epoch 14/30
1282/1282 [==============================] - 1s 843us/step - loss: 0.0040 - acc: 0.9993
Epoch 15/30
1282/1282 [==============================] - 1s 832us/step - loss: 0.0038 - acc: 0.9993 0s - loss: 0.0037 - acc: 0.999
Epoch 16/30
1282/1282 [==============================] - 1s 911us/step - loss: 0.0037 - acc: 0.9993
Epoch 17/30
1282/1282 [==============================] - 1s 809us/step - loss: 0.0036 - acc: 0.9993
Epoch 18/30
1282/1282 [==============================] - 1s 736us/step - loss: 0.0035 - acc: 0.9993
Epoch 19/30
1282/1282 [==============================] - 1s 720us/step - loss: 0.0034 - acc: 0.9993
Epoch 20/30
1282/1282 [==============================] - 1s 758us/step - loss: 0.0033 - acc: 0.9994 0s - loss: 0.0033 - acc: 0.99
Epoch 21/30
1282/1282 [============================

[0.68398633715089396,
 0.96695156695156703,
 0.53169472502805837,
 0.93828613074893796]

In [18]:
results_3 = time_series_cross_val(500, 5)
results_3

('Cross validating epoch ', 0)
(399, 500, 30)
(171, 500, 30)
(399, 500, 2)
(171, 500, 2)
Epoch 1/30
399/399 [==============================] - 3s 7ms/step - loss: 0.5299 - acc: 0.7567
Epoch 2/30
399/399 [==============================] - 1s 2ms/step - loss: 0.1987 - acc: 0.9919
Epoch 3/30
399/399 [==============================] - 1s 3ms/step - loss: 0.0850 - acc: 0.9981
Epoch 4/30
399/399 [==============================] - 1s 2ms/step - loss: 0.0442 - acc: 0.9982
Epoch 5/30
399/399 [==============================] - 1s 2ms/step - loss: 0.0276 - acc: 0.9986A: 1s - loss: 0.0314 - 
Epoch 6/30
399/399 [==============================] - 1s 2ms/step - loss: 0.0199 - acc: 0.9988
Epoch 7/30
399/399 [==============================] - 1s 2ms/step - loss: 0.0156 - acc: 0.9989
Epoch 8/30
399/399 [==============================] - 1s 2ms/step - loss: 0.0130 - acc: 0.9989
Epoch 9/30
399/399 [==============================] - 1s 2ms/step - loss: 0.0112 - acc: 0.9989
Epoch 10/30
399/399 [============

513/513 [==============================] - 1s 2ms/step - loss: 0.0055 - acc: 0.9991
Epoch 17/30
513/513 [==============================] - 1s 2ms/step - loss: 0.0053 - acc: 0.9992
Epoch 18/30
513/513 [==============================] - 1s 2ms/step - loss: 0.0051 - acc: 0.9992
Epoch 19/30
513/513 [==============================] - 1s 2ms/step - loss: 0.0049 - acc: 0.9991A: 0s - loss: 0.0058
Epoch 20/30
513/513 [==============================] - 1s 2ms/step - loss: 0.0048 - acc: 0.9992
Epoch 21/30
513/513 [==============================] - 1s 2ms/step - loss: 0.0047 - acc: 0.9992
Epoch 22/30
513/513 [==============================] - 1s 2ms/step - loss: 0.0046 - acc: 0.9992A: 0s - loss: 0.0047 - acc: 
Epoch 23/30
513/513 [==============================] - 1s 2ms/step - loss: 0.0045 - acc: 0.9992
Epoch 24/30
513/513 [==============================] - 1s 2ms/step - loss: 0.0044 - acc: 0.9992
Epoch 25/30
513/513 [==============================] - 1s 2ms/step - loss: 0.0043 - acc: 0.9992
Epoc

[0.60799849085078284, 1.0, 0.43960718294051632, 0.92482992133313457]

In [19]:
results_4 = time_series_cross_val(100, 10)
results_4

('Cross validating epoch ', 0)
(1994, 100, 30)
(855, 100, 30)
(1994, 100, 2)
(855, 100, 2)
Epoch 1/30
1994/1994 [==============================] - 2s 992us/step - loss: 0.1684 - acc: 0.9518
Epoch 2/30
1994/1994 [==============================] - 1s 447us/step - loss: 0.0145 - acc: 0.9987
Epoch 3/30
1994/1994 [==============================] - 1s 442us/step - loss: 0.0088 - acc: 0.9989
Epoch 4/30
1994/1994 [==============================] - 1s 454us/step - loss: 0.0068 - acc: 0.9991
Epoch 5/30
1994/1994 [==============================] - 1s 504us/step - loss: 0.0057 - acc: 0.9992
Epoch 6/30
1994/1994 [==============================] - 1s 532us/step - loss: 0.0050 - acc: 0.9992
Epoch 7/30
1994/1994 [==============================] - 1s 559us/step - loss: 0.0045 - acc: 0.9993
Epoch 8/30
1994/1994 [==============================] - 1s 549us/step - loss: 0.0041 - acc: 0.9993
Epoch 9/30
1994/1994 [==============================] - 1s 550us/step - loss: 0.0037 - acc: 0.9994
Epoch 10/30
1994/1

2564/2564 [==============================] - 1s 557us/step - loss: 0.0025 - acc: 0.9995 0s - loss: 0.0023 - acc
Epoch 14/30
2564/2564 [==============================] - 1s 535us/step - loss: 0.0023 - acc: 0.9995
Epoch 15/30
2564/2564 [==============================] - 1s 540us/step - loss: 0.0022 - acc: 0.9995
Epoch 16/30
2564/2564 [==============================] - 1s 532us/step - loss: 0.0020 - acc: 0.9995
Epoch 17/30
2564/2564 [==============================] - 1s 551us/step - loss: 0.0019 - acc: 0.9995
Epoch 18/30
2564/2564 [==============================] - 1s 557us/step - loss: 0.0017 - acc: 0.9996
Epoch 19/30
2564/2564 [==============================] - 1s 565us/step - loss: 0.0016 - acc: 0.9996
Epoch 20/30
2564/2564 [==============================] - 1s 557us/step - loss: 0.0015 - acc: 0.9996
Epoch 21/30
2564/2564 [==============================] - 1s 533us/step - loss: 0.0014 - acc: 0.9996
Epoch 22/30
2564/2564 [==============================] - 1s 558us/step - loss: 0.0012 - 

[0.73424237932908454,
 0.93376906318082786,
 0.60829405162738492,
 0.93075398609097126]

In [20]:
results_5 = time_series_cross_val(200, 10)
results_5

('Cross validating epoch ', 0)
(997, 200, 30)
(428, 200, 30)
(997, 200, 2)
(428, 200, 2)
Epoch 1/30
997/997 [==============================] - 2s 2ms/step - loss: 0.2894 - acc: 0.9076
Epoch 2/30
997/997 [==============================] - 1s 960us/step - loss: 0.0326 - acc: 0.9982
Epoch 3/30
997/997 [==============================] - 1s 877us/step - loss: 0.0138 - acc: 0.9987
Epoch 4/30
997/997 [==============================] - 1s 866us/step - loss: 0.0096 - acc: 0.9989
Epoch 5/30
997/997 [==============================] - 1s 860us/step - loss: 0.0077 - acc: 0.9990
Epoch 6/30
997/997 [==============================] - 1s 876us/step - loss: 0.0067 - acc: 0.9991
Epoch 7/30
997/997 [==============================] - 1s 1ms/step - loss: 0.0060 - acc: 0.9991
Epoch 8/30
997/997 [==============================] - 1s 1ms/step - loss: 0.0055 - acc: 0.9992
Epoch 9/30
997/997 [==============================] - 1s 1ms/step - loss: 0.0051 - acc: 0.9992
Epoch 10/30
997/997 [=========================

1282/1282 [==============================] - 2s 1ms/step - loss: 0.0038 - acc: 0.9994
Epoch 14/30
1282/1282 [==============================] - 2s 1ms/step - loss: 0.0036 - acc: 0.9994
Epoch 15/30
1282/1282 [==============================] - 1s 1ms/step - loss: 0.0034 - acc: 0.9994
Epoch 16/30
1282/1282 [==============================] - 2s 1ms/step - loss: 0.0033 - acc: 0.9994
Epoch 17/30
1282/1282 [==============================] - 2s 1ms/step - loss: 0.0032 - acc: 0.9994
Epoch 18/30
1282/1282 [==============================] - 2s 1ms/step - loss: 0.0030 - acc: 0.9994
Epoch 19/30
1282/1282 [==============================] - 2s 1ms/step - loss: 0.0029 - acc: 0.9994
Epoch 20/30
1282/1282 [==============================] - 2s 1ms/step - loss: 0.0028 - acc: 0.9995
Epoch 21/30
1282/1282 [==============================] - 2s 1ms/step - loss: 0.0027 - acc: 0.9995A: 0s - loss: 0.0024 
Epoch 22/30
1282/1282 [==============================] - 1s 1ms/step - loss: 0.0025 - acc: 0.9995
Epoch 23/30

[0.71563123479044233,
 0.93408427200667499,
 0.58631874298540965,
 0.91178867309952005]

In [21]:
results_6 = time_series_cross_val(500, 10)
results_6

('Cross validating epoch ', 0)
(399, 500, 30)
(171, 500, 30)
(399, 500, 2)
(171, 500, 2)
Epoch 1/30
399/399 [==============================] - 4s 9ms/step - loss: 0.5023 - acc: 0.7805A: 1s - loss: 0.5914 - acc: 0
Epoch 2/30
399/399 [==============================] - 1s 3ms/step - loss: 0.1583 - acc: 0.9962
Epoch 3/30
399/399 [==============================] - 1s 3ms/step - loss: 0.0596 - acc: 0.9981A: 0s - loss: 0.0656 - acc: 0.
Epoch 4/30
399/399 [==============================] - 1s 3ms/step - loss: 0.0295 - acc: 0.9983
Epoch 5/30
399/399 [==============================] - 1s 3ms/step - loss: 0.0184 - acc: 0.9986
Epoch 6/30
399/399 [==============================] - 1s 3ms/step - loss: 0.0136 - acc: 0.9987
Epoch 7/30
399/399 [==============================] - 1s 3ms/step - loss: 0.0110 - acc: 0.9988
Epoch 8/30
399/399 [==============================] - 1s 3ms/step - loss: 0.0094 - acc: 0.9988
Epoch 9/30
399/399 [==============================] - 1s 3ms/step - loss: 0.0083 - acc: 0.99

513/513 [==============================] - 1s 2ms/step - loss: 0.0055 - acc: 0.9992
Epoch 15/30
513/513 [==============================] - 1s 2ms/step - loss: 0.0053 - acc: 0.9992
Epoch 16/30
513/513 [==============================] - 2s 3ms/step - loss: 0.0051 - acc: 0.9993
Epoch 17/30
513/513 [==============================] - 2s 3ms/step - loss: 0.0049 - acc: 0.9993
Epoch 18/30
513/513 [==============================] - 2s 3ms/step - loss: 0.0048 - acc: 0.9993
Epoch 19/30
513/513 [==============================] - 2s 3ms/step - loss: 0.0046 - acc: 0.9993
Epoch 20/30
513/513 [==============================] - 2s 3ms/step - loss: 0.0045 - acc: 0.9993
Epoch 21/30
513/513 [==============================] - 2s 3ms/step - loss: 0.0044 - acc: 0.9993
Epoch 22/30
513/513 [==============================] - 2s 3ms/step - loss: 0.0043 - acc: 0.9993
Epoch 23/30
513/513 [==============================] - 2s 3ms/step - loss: 0.0042 - acc: 0.9993
Epoch 24/30
513/513 [==============================]

[0.62118067656878384,
 0.96296296296296291,
 0.4598540965207632,
 0.9024895427458185]